# FINM3422 - Trading Desk Analysis
## Exotic Derivatives Pricing and Portfolio Management

**Date:** May 16, 2025  
**Team:** Graduate Analysts - Exotic Derivatives Trading Desk  
**Objective:** Price and manage a portfolio of four OTC option positions

### Executive Summary
This notebook implements a comprehensive option pricing and portfolio management system for four specific OTC derivative trades:

1. **European Call on BHP** - Strike at 98% of current price, expiry Sept 15, 2027
2. **American Put on CBA** - Strike $170, expiry May 15, 2026
3. **European Up-and-In Barrier Call on WES** - Strike $80, barrier $100, expiry Sept 15, 2027
4. **European Basket Call** - Strike $175, expiry July 17, 2025, on BHP/CSL/WDS/MQG basket

The system uses object-oriented programming principles with appropriate valuation methods for each derivative type and provides comprehensive portfolio hedging analysis.

## 1. System Setup and Data Retrieval

### 1.1 Import Required Libraries and Modules
We begin by importing all necessary libraries and our custom modules for option pricing and data retrieval.

### 1.2 Market Data Retrieval

We fetch real-time market data for all required ASX stocks, calculate volatilities from historical data, estimate correlations, and obtain yield curve information. This section demonstrates our approach to sourcing market data from reliable sources.

### 1.3 Interest Rate Term Structure

Our approach uses a sophisticated yield curve boostrapping method to determine appropriate discount rates for different maturities. This ensures that we capture the term structure of interest rates accurately for each option's specific maturity.

## 2. Option Pricing Implementation

### 2.1 Trade 1: European Call Option on BHP

**Trade Details:**
- Underlying: BHP Group Ltd (BHP)
- Option Type: European Call
- Strike Price: 98% of current BHP price
- Expiry: September 15, 2027
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Black-Scholes model is most appropriate for European options as it provides a closed-form solution and is the industry standard for liquid European options.

In [4]:
from datetime import date
from Option_Classes import EuropeanOption

current_price = 39.72
strike_price = 38.9256
expiry = date(2027, 9, 15)
option_type = 'call'
today_date = date(2025, 5, 16)
sigma = 0.2272

# Instantiate the European call option
bhp_eur_call = EuropeanOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma
)

# Calculate the price
bhp_call_price = bhp_eur_call.option_price()

print(f"European Call Option on BHP (Strike = {strike_price:.2f}, Expiry = {expiry}):")
print(f"Option Price: ${bhp_call_price:.2f}")

# Calculate and print Greeks for the BHP European call option
bhp_call_greeks = bhp_eur_call.greeks()
print("\nGreeks for BHP European Call Option:") # Added a newline for better formatting
for greek_name, greek_value in bhp_call_greeks.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

European Call Option on BHP (Strike = 38.93, Expiry = 2027-09-15):
Option Price: $7.30

Greeks for BHP European Call Option:
  Delta: 0.6771
  Gamma: 0.0260
  Theta: -1.7239
  Vega: 21.7990
  Rho: 46.2770


### 2.2 Trade 2: American Put Option on CBA

**Trade Details:**
- Underlying: Commonwealth Bank of Australia (CBA)
- Option Type: American Put
- Strike Price: $170.00 (fixed)
- Expiry: May 15, 2026
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** Binomial tree model is used for American options as it can handle the early exercise feature, which is valuable for puts when the stock price falls significantly below the strike.

In [3]:
from datetime import date
from Option_Classes import AmericanOption

# Trade parameters
current_price = 169.66
strike_price = 170.00
expiry = date(2026, 5, 15)
option_type = 'put'
today_date = date(2025, 5, 16)
sigma = 0.18055
n_steps = 1000

# Instantiate the American put option
cba_american_put = AmericanOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma,
    n_steps=n_steps
)

# Calculate the price
cba_put_price = cba_american_put.option_price()

print(f"American Put Option on CBA (Strike = {strike_price:.2f}, Expiry = {expiry}):")
print(f"Option Price: ${cba_put_price:.2f}")

# Calculate and print Greeks for the CBA American put option
cba_put_greeks = cba_american_put.greeks()
print("\nGreeks for CBA American Put Option:") # Added a newline for better formatting
for greek_name, greek_value in cba_put_greeks.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

American Put Option on CBA (Strike = 170.00, Expiry = 2026-05-15):
Option Price: $10.05

Greeks for CBA American Put Option:
  Delta: -0.4331
  Gamma: 0.0056
  Theta: -3.9196
  Vega: 64.9070
  Rho: -53.6915


### 2.3 Trade 3: European Up-and-In Barrier Call on WES

**Trade Details:**
- Underlying: Wesfarmers Limited (WES)
- Option Type: European Up-and-In Barrier Call
- Strike Price: $80.00
- Barrier Level: $100.00 (up-and-in)
- Expiry: September 15, 2027
- Position: **BUY** (Bank purchases the option)

**Valuation Method:** We use both binomial tree and Monte Carlo methods for barrier options. The binomial method provides discrete barrier monitoring while Monte Carlo offers continuous monitoring simulation.

In [2]:
from datetime import date
from Option_Classes import BarrierOption

# Trade 3 parameters - European Up-and-In Barrier Call on WES
# Note: current_price and sigma for WES should be updated with actual market data.
current_price = 82.56  # Placeholder WES price - REPLACE WITH ACTUAL
strike_price = 80.00
barrier_price = 100.00
expiry = date(2027, 9, 15)
option_type = 'call'
today_date = date(2025, 5, 16)
sigma = 0.1839 # Placeholder WES volatility - REPLACE WITH ACTUAL
n_steps_binomial = 1000 # Number of steps for binomial model
n_steps_mc = 100      # Number of time steps for Monte Carlo simulation (for price)
                      # The greeks method for BarrierOption also uses n_steps for its internal pricing calls.

# Instantiate the barrier option (Binomial for primary pricing)
wes_barrier_call_binomial = BarrierOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma,
    barrier_price=barrier_price,
    method="binomial", # Using binomial for the main price and Greeks
    n_steps=n_steps_binomial
)

# Calculate the price using binomial method
wes_barrier_price_binomial_val = wes_barrier_call_binomial.option_price()

print(f"European Up-and-In Barrier Call on WES (Strike = {strike_price:.2f}, Barrier = {barrier_price:.2f}, Expiry = {expiry}):")
print(f"Option Price (Binomial): ${wes_barrier_price_binomial_val:.2f}")

# Calculate and print Greeks using the binomial model instance
# The .greeks() method in BarrierOption will use the 'method' and 'n_steps' it was initialized with.
wes_barrier_greeks_binomial = wes_barrier_call_binomial.greeks()
print("\nGreeks for WES Barrier Call (Binomial Model):")
for greek_name, greek_value in wes_barrier_greeks_binomial.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

# For Monte Carlo pricing (as per original notebook structure)
wes_barrier_call_mc = BarrierOption(
    current_price=current_price,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigma,
    barrier_price=barrier_price,
    method="monte-carlo",
    n_steps=n_steps_mc # n_steps for MC price simulation, can differ from binomial
)
wes_barrier_price_mc_val = wes_barrier_call_mc.option_price()
print(f"\nOption Price (Monte Carlo): ${wes_barrier_price_mc_val:.2f}")

# If you also want Greeks from the Monte Carlo perspective (will be slower):
wes_barrier_greeks_mc = wes_barrier_call_mc.greeks()
print("\nGreeks for WES Barrier Call (Monte Carlo Model):")
for greek_name, greek_value in wes_barrier_greeks_mc.items():
    print(f"  {greek_name.capitalize()}: {greek_value:.4f}")


European Up-and-In Barrier Call on WES (Strike = 80.00, Barrier = 100.00, Expiry = 2027-09-15):
Option Price (Binomial): $13.72

Greeks for WES Barrier Call (Binomial Model):
  Delta: 0.6974
  Gamma: -0.0000
  Theta: -3.2244
  Vega: 43.8444
  Rho: 104.9407

Option Price (Monte Carlo): $12.76

Greeks for WES Barrier Call (Monte Carlo Model):
  Delta: 39.2693
  Gamma: 3845.6263
  Theta: 104.1706
  Vega: 74.0920
  Rho: 116.6325


### 2.4 Trade 4: European Basket Call Option

**Trade Details:**
- Underlying Basket: 10% BHP, 35% CSL, 15% WDS, 40% MQG
- Option Type: European Call on basket value
- Strike Price: $175.00
- Expiry: July 17, 2025
- Position: **WRITE** (Bank sells the option)

**Valuation Method:** Monte Carlo simulation with correlation structure is essential for basket options as there's no closed-form solution. We use Cholesky decomposition to generate correlated asset price paths.

In [6]:
from datetime import date
import numpy as np
from Option_Classes import BasketOption # Ensure BasketOption is imported

# Trade 4 parameters - European Basket Call Option
# Basket definition: 10% BHP, 35% CSL, 15% WDS, 40% MQG
asset_tickers_in_basket = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = [0.10, 0.35, 0.15, 0.40] # Corresponding to asset_tickers_in_basket

# Current prices for the basket assets
# These should ideally be dynamic or updated regularly.
# Using the last known values from data.py output for this example.
current_prices = [39.72, 241.82, 21.92, 211.25] # BHP, CSL, WDS, MQG

strike_price = 175.00
expiry = date(2025, 7, 17)
option_type = 'call'
today_date = date(2025, 5, 16)

# Volatilities for assets in asset_tickers_in_basket (BHP, CSL, WDS, MQG)

sigma_bhp = 0.2064  
sigma_csl = 0.2109   
sigma_wds = 0.2637    
sigma_mqg = 0.2085    
sigmas = [sigma_bhp, sigma_csl, sigma_wds, sigma_mqg] # List of volatilities for the basket

# Hardcoded Correlation Matrix for BHP, CSL, WDS, MQG
# Order: BHP with (BHP, CSL, WDS, MQG), CSL with (BHP, CSL, WDS, MQG), ...
correlation_matrix_np = np.array([
    [1.000000, 0.452170, 0.807649, 0.878665],  # BHP with BHP, CSL, WDS, MQG
    [0.452170, 1.000000, 0.581252, 0.243513],  # CSL with BHP, CSL, WDS, MQG
    [0.807649, 0.581252, 1.000000, 0.651679],  # WDS with BHP, CSL, WDS, MQG
    [0.878665, 0.243513, 0.651679, 1.000000]   # MQG with BHP, CSL, WDS, MQG
])

# Instantiate the basket option
basket_call = BasketOption(
    current_prices=current_prices,
    weights=weights,
    strike_price=strike_price,
    expiry=expiry,
    option_type=option_type,
    today_date=today_date,
    sigma=sigmas, 
    correlation_matrix=correlation_matrix_np # Using the hardcoded NumPy array
)

# Calculate the price
basket_call_price = basket_call.option_price()

print(f"European Basket Call Option (Strike = {strike_price:.2f}, Expiry = {expiry}):")
print(f"Basket composition: {weights[0]*100:.0f}% BHP, {weights[1]*100:.0f}% CSL, {weights[2]*100:.0f}% WDS, {weights[3]*100:.0f}% MQG")
asset_price_info = ", ".join([f"{asset.replace('.AX','')}: ${price:.2f}" for asset, price in zip(asset_tickers_in_basket, current_prices)])
print(f"Using current prices: {asset_price_info}") # Changed "latest" to "current" as they are now hardcoded in this cell
print(f"Option Price: ${basket_call_price:.2f}")

# Calculate and print Greeks for the Basket option
basket_call_greeks = basket_call.greeks()
print("\nGreeks for European Basket Call Option:")
for greek_name, greek_value in basket_call_greeks.items():
    if greek_name == 'deltas': 
        print(f"  Deltas (for {', '.join(asset.replace('.AX','') for asset in asset_tickers_in_basket)} respectively):")
        for i, delta_val in enumerate(greek_value):
            print(f"    {asset_tickers_in_basket[i].replace('.AX','')}: {delta_val:.4f}")
    else:
        print(f"  {greek_name.capitalize()}: {greek_value:.4f}")

European Basket Call Option (Strike = 175.00, Expiry = 2025-07-17):
Basket composition: 10% BHP, 35% CSL, 15% WDS, 40% MQG
Using current prices: BHP: $39.72, CSL: $241.82, WDS: $21.92, MQG: $211.25
Option Price: $6.10

Greeks for European Basket Call Option:
  Deltas (for BHP, CSL, WDS, MQG respectively):
    BHP: 1.9480
    CSL: -1.2514
    WDS: -2.9398
    MQG: 2.1189
  Vega: 22.8157
  Theta: -35.6431
  Rho: 17.0699
  Portfolio_volatility: 0.1755


## 3. Portfolio Analysis and Risk Management

### 3.1 Portfolio Construction

We now construct the complete portfolio with all four trades, considering the bank's position (long/short) for each option. This section demonstrates sophisticated portfolio management techniques for exotic derivatives.

### 3.2 Comprehensive Risk Analysis

This section provides detailed risk analysis including individual option contributions and portfolio-level risk metrics. Our analysis covers all major Greeks and their implications for portfolio management.

### 3.3 Hedging Strategy and Recommendations

This section provides comprehensive hedging recommendations based on the portfolio's Greek exposures. We analyse both basic delta hedging and more sophisticated multi-dimensional hedging strategies appropriate for an institutional trading desk.

### 3.4 Portfolio Stress Testing and Scenario Analysis

We conduct scenario analysis to understand how the portfolio performs under different market conditions. This demonstrates sophisticated risk management practices expected in institutional environments.

## 4. Summary and Trading Recommendations

### 4.1 Executive Summary

This section provides a comprehensive summary of all pricing results, risk assessments, and strategic recommendations for the trading desk.

## 5. Technical Documentation

### 5.1 Model Validation and Implementation Notes

This section documents the technical implementation decisions and validates our modeling choices against industry best practices.